In [1]:
using Revise
using PastaQ
using ITensors
using Random
using OptimKit
using Zygote
using Zygote: ChainRulesCore
using BenchmarkTools
using LinearAlgebra
using JLD2
using Flux

import mVQE
using mVQE.Hamiltonians: hamiltonian_tfi, hamiltonian_ghz
using mVQE.ITensorsExtension: projective_measurement
using mVQE: loss, optimize_and_evolve
using mVQE.Circuits: VariationalCircuitRy, VariationalMeasurement, VariationalMeasurementMC, VariationalMeasurementMCFeedback
using mVQE.Misc: get_ancillas_indices, pprint, unravel_index
using mVQE.Optimizers: OptimizerWrapper

┌ Info: Precompiling PastaQ [30b07047-aa8b-4c78-a4e8-24d720215c19]
└ @ Base loading.jl:1664
┌ Info: Precompiling Zygote [e88e6eb3-aa80-5325-afca-941959d7151f]
└ @ Base loading.jl:1664
┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1664
┌ Info: Precompiling mVQE [fbf8b4f3-d5ee-4fcd-97a7-1cb357585aed]
└ @ Base loading.jl:1664


In [2]:
state_indices, ancillas_indices, N = get_ancillas_indices(6, 1)

# build MPO "cost function"
hilbert = qubits(N)

H = MPO(hamiltonian_tfi(state_indices, 0.1), hilbert)
#H = hamiltonian_ghz(state_indices, hilbert)

# VQE
Random.seed!(1)

# Initialize state
ψ = productstate(hilbert, fill(0, N))
ρs = outer(ψ, ψ');

In [ ]:
vmodels = [VariationalCircuitRy(N, 2) for _ in 1:1]
model2 = VariationalMeasurementMC(vmodels, ancillas_indices)
l, grad = @time mVQE.loss_and_grad(ψ, H, model2, 1)
l, grad[grad.params[1]]

In [8]:
using Pkg
Pkg.project()

Pkg.API.ProjectInfo("mVQE", UUID("fbf8b4f3-d5ee-4fcd-97a7-1cb357585aed"), v"0.1.0", true, Dict{String, Base.UUID}("Statistics" => UUID("10745b16-79ce-11e8-11f9-7d13ad32a3b2"), "Revise" => UUID("295af30f-e4ad-537b-8983-00126c2a3abe"), "Random" => UUID("9a3f8284-a2c9-5f02-9a11-845980a1fd5c"), "ParallelDataTransfer" => UUID("2dcacdae-9679-587a-88bb-8b444fb7085b"), "Suppressor" => UUID("fd094767-a336-5f1f-9728-57cf17d0bbfb"), "ITensors" => UUID("9136182c-28ba-11e9-034c-db9fb085ebd5"), "Distributed" => UUID("8ba89e20-285c-5b6f-9357-94700520ee1b"), "PastaQ" => UUID("30b07047-aa8b-4c78-a4e8-24d720215c19"), "LinearAlgebra" => UUID("37e2e46d-f89d-539d-b4ee-838fcccc9c8e"), "PyCall" => UUID("438e738f-606a-5dbb-bf0a-cddfbfd45ab0")…), "/local/alcalde/workprojects/mVQE/mVQE/Project.toml")